In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
import os
sys.path.append(os.path.expanduser('~'))
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter

# Cargue de Queries y parametros

In [2]:
city = 'SPO'

In [3]:
# Parameters
city = "CWB"


In [4]:
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=200))#.strftime('%Y-%m-%d') #1 months

In [5]:
print(os.getcwd())

/home/jupyter-felipe.guevara


In [6]:
margenes = pd.read_csv("/home/jupyter-felipe.guevara/Growth/CLTV/Margenes_neto.csv")
retention = pd.read_csv("/home/jupyter-felipe.guevara/Growth/CLTV/retention_year.csv")

In [7]:
retention_aux =  retention.copy()

for i,ret in enumerate(retention_aux.values):
    retention_aux.iloc[i,2:] = [(1-(j-ret[-1])/100) if j-ret[-1] >= 0 else 1 for j in ret[2:]]

In [8]:
clear_buyers = get_fresh_query_result("https://internal-redash.federate.frubana.com/",138674,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [9]:
buyers_11 = get_fresh_query_result("https://internal-redash.federate.frubana.com/",212712,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [10]:
query = """
SELECT DISTINCT

c.assignee_ref_id AS "Owner",
cm.member_ref_id AS "customer_id",
--cm.status AS "Estado de la asignacion",
--c.type as "Tipo de asociacion",
--c.flow_type AS "Tipo de carga",
z.value AS "Ciudad"
--p.since AS "Semana"

FROM postgres_growth_offline."growth_clustering.clusters"               c
INNER JOIN postgres_growth_offline."growth_clustering.cluster_members"  cm  ON cm.cluster_id = c.id
INNER JOIN postgres_growth_offline."growth_clustering.periods"          p   ON p.id = c.period_id
INNER JOIN postgres_growth_offline."growth_clustering.zones"            z   ON z.id = c.zone_id

where p.since = DATE_TRUNC('week', date(getdate()) - interval '0 week')
and z.value in ('BOG','BAQ','MDE','CMX','SPO','BHZ','CWB','VCP')
and c.type = 'FARMER_OFFLINE'
and c.status = 'ACTIVE'
and c.enabled = 'true'
"""
data_farming = read_connection_data_warehouse.runQuery(query)
data_farming[["customer_id"]] = data_farming[["customer_id"]].astype(int)

In [11]:
# Define a function to run the query
def run_query_with_retry(query):
    max_retries = 5
    retries = 0
    while retries < max_retries:
        try:
            # Attempt to execute the query
            data_ventas_aux = run_read_dwd_query(query)
            data_ventas_aux[["gmv_usd", "cant"]] = data_ventas_aux[["gmv_usd", "cant"]].astype(float)
            data_ventas_aux['submit_date'] = pd.to_datetime(data_ventas_aux['submit_date'])
            return data_ventas_aux  # If successful, return the result
        except Exception as e:
            retries += 1
            print(f"An error occurred: {str(e)}")
            print(f"Retrying... ({retries}/{max_retries})")
    # If max retries reached without success, raise an error
    raise RuntimeError("Failed to execute the query after multiple retries")

In [12]:
query = """
SELECT 
    s.identifier_value AS city,
    dc.source_id AS customer_id,
    DATE(fs.order_submitted_date) AS submit_date,
    fs.order_id,
    dp.card_id AS padre_sku_id,
    cat.parent_description AS cat,
    CASE
        WHEN s.identifier_value = 'CMX' THEN fs.gmv_pxq_local/19.65
        WHEN s.identifier_value IN ('SPO','BHZ','CWB','VCP') THEN fs.gmv_pxq_local/4.75
        WHEN s.identifier_value IN ('BOG','BAQ','MDE') THEN fs.gmv_pxq_local/3776
    ELSE fs.gmv_pxq_local 
    END AS gmv_usd,
    fs.product_quantity_x_step_unit AS cant

FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND DATE(fs.order_submitted_date) > current_date - 730
    AND s.identifier_value = '{city}'
    AND dc.source_id IN {clientes}
""".format(clientes=tuple(set(clear_buyers[clear_buyers.registered_city == city].customer_id.unique())), city = city)

# Call the function with your query
data_ventas_aux = run_query_with_retry(query)

In [13]:
data_ventas_aux['cat'] = data_ventas_aux['cat'].replace('Frutas e verduras', 'Frutas e Verduras')
data_ventas_aux['cat'] = data_ventas_aux['cat'].replace('Laticínios e ovos', 'Laticínios e Ovos')
data_ventas_aux['cat'] = data_ventas_aux['cat'].replace('Limpeza e higiene', 'Limpeza e Higiene')
data_ventas_aux['cat'] = data_ventas_aux['cat'].replace('Carnes, aves e peixes', 'Carnes, Aves e Peixes')
data_ventas_aux['cat'] = data_ventas_aux['cat'].replace('Abarrotes & despensa', 'Abarrotes')
data_ventas_aux['cat'] = data_ventas_aux['cat'].replace('Aseo e higiene', 'Aseo e Higiene')
data_ventas_aux['cat'] = data_ventas_aux['cat'].replace('Frutas & verduras', 'Frutas & Verduras')
data_ventas_aux['cat'] = data_ventas_aux['cat'].replace('Carne, pollo & pescados', 'Carne, Pollo & Pescados')
data_ventas_aux['cat'] = data_ventas_aux['cat'].replace('Lácteos & huevos', 'Lácteos & Huevos')

In [14]:
check_df = data_ventas_aux.groupby(by=['customer_id']).agg({'submit_date':np.max}).reset_index()
df = data_ventas_aux[data_ventas_aux.customer_id.isin(check_df[check_df.submit_date >= customer_date_desde].customer_id.unique())].groupby(by=['city', 'customer_id', 'submit_date','cat']).agg({'gmv_usd':'sum','cant':'sum'}).reset_index()

# Generacion de df con info by cat by client

In [15]:
df = df.sort_values('submit_date',ignore_index=True)

#fix for id problems and taking into account only dates
df['order_id']= df.submit_date.apply(lambda x:x.toordinal()) 

#Days of difference between orders for customer and for cat
df['days_bet'] = (df.submit_date -df.groupby('customer_id').submit_date.transform('min')).dt.days

In [16]:
#This calculate in res the difference between consecutive orders and with the agg we calculate all metrics
df['res'] = df.groupby(by=['customer_id', 'cat'])['days_bet'].diff()
df_grouped = df.groupby(by=['city', 'customer_id', 'cat']).agg({'res': [np.min, np.max, np.mean, np.median, np.std],
                                                                    'order_id': [lambda x: x.nunique()], 
                                                                    'gmv_usd': [np.mean],
                                                                    'cant': [np.mean],
                                                                    'submit_date': [np.max, lambda x: (x.max()-x.min()).days, lambda x: (datetime.datetime.today() - x.max()).days, np.min]
                                                                    }).reset_index()

In [17]:
#Finally we define everything in a df2 for further processing
df2 = df_grouped.copy()

df2.columns = ['city', 'customer_id', 'cat', 'min_diff_days', 'max_diff_days', 'avg_diff_days', 'median_diff_days', 'stdev_diff_days',
               'num_orders', 'aov_cat_usd', 'avg_cant_cat', 'last_order_date', 'diff_from_last_to_first_order', 'days_since_last_order','first_order_date']

In [18]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

In [19]:
def cat_metrics_2(cat):
    df_cat = df2[df2.cat == cat].copy()
    
    df_cat['adjust_std'] = pd.qcut(df_cat.stdev_diff_days, 10, labels=[1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1])
    df_cat[["adjust_std"]] = df_cat[["adjust_std"]].astype(float)
    
    df_cat['adjust_cohort'] = [1 if ((x[13]) - (x[5]+x[7]) <= 0) else  
                 1/(((x[13]) - x[5])/x[7]) if x[7] != 0 else
                 1/(((x[13]) - x[5])/np.round(1+(x[5]*0.7)))
                 for x in df_cat.values]
    
    df_cat["margen"] = [margenes[(margenes.category == x[2]) & (margenes.region_code == x[0])]["net_margin"].unique()[0]/100 for x in df_cat.values]
    #mirar aca
    df_cat['adjust_retention'] = [1 if diff_month(x[11],x[14])+1 >= 10 else 
                                  retention_aux[(retention_aux.category == x[2]) & (retention_aux.region_code == x[0])][str(diff_month(x[11],x[14])+1)].unique()[0] for x in df_cat.values]
    
    df_cat['median_diff_days_adjusted'] = 365/df_cat.median_diff_days
    
    df_cat["churned"] = [0 if ((x[13]) - (x[5]+x[7]*2) <= 0) else 1 for x in df_cat.values]
    
    df_cat[['avg_diff_days','stdev_diff_days']] = df_cat[['avg_diff_days','stdev_diff_days']].fillna(0)

    df_cat['churned_date'] = [(x[11] + datetime.timedelta(days=int(x[5]+x[7]*2))).strftime('%Y-%m-%d') for x in df_cat.values] #1 months
    
    df_cat["CLTV"] = df_cat.median_diff_days_adjusted * df_cat.margen * df_cat.aov_cat_usd * df_cat.adjust_retention * df_cat.adjust_std * df_cat.adjust_cohort
    df_cat["CLTV_sin_margen"] = df_cat.median_diff_days_adjusted * df_cat.aov_cat_usd * df_cat.adjust_retention * df_cat.adjust_std * df_cat.adjust_cohort
    df_cat["CLTV_sin_churn"] = df_cat.median_diff_days_adjusted * df_cat.margen * df_cat.aov_cat_usd * df_cat.adjust_retention * df_cat.adjust_std
    
    df_cat['cash_margin'] = df_cat.aov_cat_usd * df_cat.margen

    return df_cat

In [20]:
df3 = pd.DataFrame()

for cat in df2.cat.unique():
   print(cat)
   if cat not in ('Frubana prime','¡productos nuevos!','No value','Mesa & cocina','Insumos internos cat','Despensa'):
      df3 = df3.append(cat_metrics_2(cat))
   print(f'Done {cat}')

Bebidas


Done Bebidas
Carnes, Aves e Peixes


Done Carnes, Aves e Peixes
Congelados


Done Congelados
Frutas e Verduras


Done Frutas e Verduras
Laticínios e Ovos


Done Laticínios e Ovos
Limpeza e Higiene


Done Limpeza e Higiene
Mercearia


Done Mercearia
Descartáveis


Done Descartáveis


In [21]:
df3.head(5)

,city,customer_id,cat,min_diff_days,max_diff_days,avg_diff_days,median_diff_days,stdev_diff_days,num_orders,aov_cat_usd,...,adjust_cohort,margen,adjust_retention,median_diff_days_adjusted,churned,churned_date,CLTV,CLTV_sin_margen,CLTV_sin_churn,cash_margin
0,CWB,109267474,Bebidas,16.0,369.0,91.166667,38.0,136.926866,7,14.851128,...,1.000000,0.148,1.000,9.605263,0,2025-01-06,2.111205,14.264899,2.111205,2.197967
7,CWB,109816902,Bebidas,NaN,NaN,0.000000,NaN,0.000000,1,21.675789,...,NaN,0.148,0.735,NaN,1,2023-06-14,NaN,NaN,NaN,3.208017
13,CWB,109817681,Bebidas,5.0,315.0,38.647059,17.0,73.555201,18,13.513684,...,1.000000,0.148,1.000,21.470588,0,2025-01-02,8.588344,58.029349,8.588344,2.000025
26,CWB,109823370,Bebidas,1.0,28.0,2.147619,1.0,2.432263,211,31.169229,...,0.020294,0.148,1.000,365.000000,1,2024-03-13,34.169963,230.878130,1683.761758,4.613046
34,CWB,110354798,Bebidas,3.0,64.0,13.934783,10.5,11.693117,47,101.378634,...,0.174354,0.148,1.000,34.761905,1,2024-05-23,72.750279,491.555936,417.255146,15.004038


# Aca empieza el analisis global by customer 

In [22]:
#Here, i group the df3 (the df with the whole info by cat to customer lvl)
df_new3 = df3[['city','customer_id','aov_cat_usd','cash_margin','CLTV_sin_churn','churned']].groupby(by=['city','customer_id']).sum().reset_index()

In [23]:
#The processing data for customer level is made here

df_new = df.groupby(by=['city','customer_id','order_id','submit_date']).sum().reset_index()

df_new['days_bet'] = (df_new.submit_date -df_new.groupby('customer_id').submit_date.transform('min')).dt.days

#This calculate in res the difference between consecutive orders and with the agg we calculate all metrics
df_new['res'] = df_new.groupby(by=['customer_id'])['days_bet'].diff()
df_grouped_2 = df_new.groupby(by=['city', 'customer_id']).agg({'res': [np.min, np.max, np.mean, np.median, np.std],
                                                                    'order_id': [lambda x: x.nunique()], 
                                                                    'gmv_usd': [np.mean],
                                                                    'cant': [np.mean],
                                                                    'submit_date': [np.max, lambda x: (x.max()-x.min()).days, lambda x: (datetime.datetime.today() - x.max()).days, np.min]
                                                                    }).droplevel(0, axis="columns").reset_index()

In [24]:
#Finally we define everything in a df_new2 at level customer for further processing
df_new2 = df_grouped_2.copy()

df_new2.columns = ['city', 'customer_id', 'min_diff_days', 'max_diff_days', 'avg_diff_days', 'median_diff_days', 'stdev_diff_days',
               'num_orders', 'aov_ALL_usd', 'ALL_avg_cant', 'last_order_date', 'diff_from_last_to_first_order', 'days_since_last_order','first_order_date']

In [25]:
#Finally we calculate the next parameters and also integred with df_new3 where we can have the info grouped by customer about the category performance
df_new2["churned_ALL"] = [0 if ((x[12]) - (x[4]+x[6]*2) <= 0) else 1 for x in df_new2.values]

df_new2['adjust_std'] = pd.qcut(df_new2.stdev_diff_days, 10, labels=[1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1])
df_new2[["adjust_std"]] = df_new2[["adjust_std"]].astype(float)

df_new2['adjust_retention'] = [1 if diff_month(x[10],x[13])+1 >= 10 else 
                                retention_aux[(retention_aux.region_code == x[0])][str(diff_month(x[10],x[13])+1)].max() for x in df_new2.values]

df_new2['median_diff_days_adjusted'] = 365/df_new2.median_diff_days

#this merge is to include the info for the total cash_margin, how many cats is churned and the total cat CLTV
df_new2 = df_new2.merge(df_new3[['customer_id','cash_margin','CLTV_sin_churn','churned']], on='customer_id', how='left')

In [26]:
#We calculate now the CLTV taking into account the global behavior of the customer
df_new2["ALL_CLTV_sin_churn"] = df_new2.median_diff_days_adjusted * df_new2.adjust_retention * df_new2.adjust_std * df_new2.cash_margin / 2
df_new2[['CLTV_sin_churn','ALL_CLTV_sin_churn']] = df_new2[['CLTV_sin_churn','ALL_CLTV_sin_churn']].fillna(0)

#df_new2["CLTV"] = df_new2.median_diff_days_adjusted * df_new2.margen * df_new2.aov_cat_usd * df_new2.adjust_retention * df_new2.adjust_std * df_new2.adjust_cohort
#df_new2["CLTV_sin_margen"] = df_new2.median_diff_days_adjusted * df_new2.aov_cat_usd * df_new2.adjust_retention * df_new2.adjust_std * df_new2.adjust_cohort

#If the Total Cat CLTV is NA because it is so few orders for each category, we create a Adjust_CLTV to include the global or ALL_CLTV_sin_churn instead of the NULL value
df_new2['Adjust_CLTV'] = [x[21] if x[19] == 0 else x[19] for x in df_new2.values]

print(df_new2.Adjust_CLTV.sum(),df_new2.CLTV_sin_churn.sum())

2470980.415470629 2467940.9538048953


In [27]:
#Finally, we calculate the ALL_churned_date based on the general behavior of the customer
df_new2[['avg_diff_days','stdev_diff_days']] = df_new2[['avg_diff_days','stdev_diff_days']].fillna(0)
df_new2['ALL_churned_date'] = [(x[10] + datetime.timedelta(days=int(x[4]+x[6]*2))).strftime('%Y-%m-%d') for x in df_new2.values] #1 months

# Final processing of the file to export

In [28]:
#We will merge the df_new2 with the clear_buyers in order to filter by restaurants and take away kams (also to include the microsegment)
df_new2 = df_new2.merge(clear_buyers[clear_buyers.registered_city == city][['customer_id','business_type','microsegment_name','is_kam']], on='customer_id', how='left')
df_new2_end = df_new2[(df_new2.is_kam == 0)].copy() #df_new2[(df_new2.business_type == 'Restaurante') & (df_new2.is_kam == 0)].copy()

In [29]:
#This cell is to calculate the cumulative sum to create the tiers further
df_new2_end = df_new2_end.sort_values(['Adjust_CLTV'], ascending=[False])
df_new2_end['cumsum'] = df_new2_end.groupby('city')['Adjust_CLTV'].cumsum()
df_new2_end['run_pct'] = df_new2_end.groupby('city')['Adjust_CLTV'].apply(lambda x: (x/x.sum()).cumsum())

In [30]:
#This conditional create the tiers
df_new2_end['tier'] = ['A' if x <= 0.1 else
                       'B' if x <= 0.2 else
                       'C' if x <= 0.3 else
                       'D' if x <= 0.4 else
                       'E' if x <= 0.5 else
                       'F' if x <= 0.6 else
                       'G' if x <= 0.7 else
                       'H' if x <= 0.8 else
                       'I' if x <= 0.9 else
                       'J' for x in df_new2_end.run_pct.values]

In [31]:
df_new2_end.tier.value_counts()

J    1525
I     427
H     268
G     191
F     150
E     117
D      92
C      70
B      51
A      28
Name: tier, dtype: int64

In [32]:
df_new2_end.rename(columns={'num_orders': 'ALL_num_orders', 'CLTV_sin_churn': 'TOTAL_CAT_CLTV_sin_churn'}, inplace=True)

df3.rename(columns={'num_orders': 'num_orders_cat', 'churned': 'churned_cat','CLTV_sin_churn':'CAT_CLTV_sin_churn'}, inplace=True)

In [33]:
df_new2_end.columns

Index(['city', 'customer_id', 'min_diff_days', 'max_diff_days',
       'avg_diff_days', 'median_diff_days', 'stdev_diff_days',
       'ALL_num_orders', 'aov_ALL_usd', 'ALL_avg_cant', 'last_order_date',
       'diff_from_last_to_first_order', 'days_since_last_order',
       'first_order_date', 'churned_ALL', 'adjust_std', 'adjust_retention',
       'median_diff_days_adjusted', 'cash_margin', 'TOTAL_CAT_CLTV_sin_churn',
       'churned', 'ALL_CLTV_sin_churn', 'Adjust_CLTV', 'ALL_churned_date',
       'business_type', 'microsegment_name', 'is_kam', 'cumsum', 'run_pct',
       'tier'],
      dtype='object')

In [34]:
#We merge with data_farming so we get the info farming for each customer
df_new2_end = df_new2_end[['customer_id','ALL_num_orders', 'aov_ALL_usd', 'churned_ALL', 'ALL_churned_date','TOTAL_CAT_CLTV_sin_churn',
       'ALL_CLTV_sin_churn', 'Adjust_CLTV','tier', 'microsegment_name', 'business_type', 'is_kam']].merge(data_farming[data_farming.ciudad == city][['customer_id','owner']], on='customer_id', how='left')
df_new2_end[['owner']] = df_new2_end[['owner']].fillna('No Farming')

In [35]:
df_new2_end.rename(columns={'num_orders': 'ALL_num_orders', 'CLTV_sin_churn': 'TOTAL_CAT_CLTV_sin_churn'}, inplace=True)

df3.rename(columns={'num_orders': 'num_orders_cat', 'churned': 'churned_cat','CLTV_sin_churn':'CAT_CLTV_sin_churn'}, inplace=True)

In [36]:
df_final = df3[['city', 'customer_id', 'cat', 'min_diff_days', 'max_diff_days', 'avg_diff_days', 'median_diff_days', 'stdev_diff_days', 'num_orders_cat',
       'aov_cat_usd','last_order_date', 'diff_from_last_to_first_order', 'days_since_last_order',
       'first_order_date', 'adjust_std', 'adjust_cohort','adjust_retention', 
       'median_diff_days_adjusted', 'churned_cat', 'churned_date','CAT_CLTV_sin_churn']].merge(df_new2_end, on='customer_id', how='inner')

In [37]:
df_final.head(10)

,city,customer_id,cat,min_diff_days,max_diff_days,avg_diff_days,median_diff_days,stdev_diff_days,num_orders_cat,aov_cat_usd,...,churned_ALL,ALL_churned_date,TOTAL_CAT_CLTV_sin_churn,ALL_CLTV_sin_churn,Adjust_CLTV,tier,microsegment_name,business_type,is_kam,owner
0,CWB,109267474,Bebidas,16.0,369.0,91.166667,38.0,136.926866,7,14.851128,...,0,2024-09-15,244.100941,181.075909,244.100941,J,Sucos/Açaí,Restaurante,False,No Farming
1,CWB,109267474,"Carnes, Aves e Peixes",75.0,75.0,75.000000,75.0,0.000000,2,27.210526,...,0,2024-09-15,244.100941,181.075909,244.100941,J,Sucos/Açaí,Restaurante,False,No Farming
2,CWB,109267474,Congelados,64.0,64.0,64.000000,64.0,0.000000,2,4.593684,...,0,2024-09-15,244.100941,181.075909,244.100941,J,Sucos/Açaí,Restaurante,False,No Farming
3,CWB,109267474,Frutas e Verduras,1.0,282.0,8.987654,3.0,31.897294,82,25.644262,...,0,2024-09-15,244.100941,181.075909,244.100941,J,Sucos/Açaí,Restaurante,False,No Farming
4,CWB,109267474,Laticínios e Ovos,1.0,375.0,24.071429,7.0,69.502959,29,14.522250,...,0,2024-09-15,244.100941,181.075909,244.100941,J,Sucos/Açaí,Restaurante,False,No Farming
5,CWB,109267474,Limpeza e Higiene,4.0,368.0,73.222222,28.0,116.300019,10,6.222105,...,0,2024-09-15,244.100941,181.075909,244.100941,J,Sucos/Açaí,Restaurante,False,No Farming
6,CWB,109267474,Mercearia,1.0,289.0,13.716981,6.0,39.938596,54,12.411072,...,0,2024-09-15,244.100941,181.075909,244.100941,J,Sucos/Açaí,Restaurante,False,No Farming
7,CWB,109816902,Bebidas,NaN,NaN,0.000000,NaN,0.000000,1,21.675789,...,0,2024-07-13,346.058593,368.475660,346.058593,J,Grill/Churrascaria/Espetinho,Restaurante,False,No Farming
8,CWB,109816902,Congelados,NaN,NaN,0.000000,NaN,0.000000,1,15.551579,...,0,2024-07-13,346.058593,368.475660,346.058593,J,Grill/Churrascaria/Espetinho,Restaurante,False,No Farming
9,CWB,109816902,Frutas e Verduras,1.0,27.0,8.814815,7.0,4.561006,82,32.040873,...,0,2024-07-13,346.058593,368.475660,346.058593,J,Grill/Churrascaria/Espetinho,Restaurante,False,No Farming


In [38]:
#df_final.to_excel(f"CLTV_{city}.xlsx",index=False)

# Info for SAC team

In [39]:
df_SAC = df_final[['city','customer_id','tier','Adjust_CLTV']].drop_duplicates()

In [40]:
letter_to_number = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10}
df_SAC['tier'] = df_SAC['tier'].map(letter_to_number)

In [41]:
# Iterate through the customer_ids
for customer_id in buyers_11.loc[buyers_11.region_code == city].uid.unique():
    # Check if customer_id is already in the DataFrame
    if customer_id in df_SAC['customer_id'].values:
        # If yes, change its tier to 11
        df_SAC.loc[df_SAC['customer_id'] == customer_id, 'tier'] = 11
    else:
        # If not, add a new row with tier 11
        df_SAC = df_SAC.append({'city': city, 'customer_id': customer_id, 'tier': 11, 'Adjust_CLTV':1}, ignore_index=True)

In [42]:
df_SAC.to_csv(f"/home/jupyter-felipe.guevara/Growth/CLTV/{city}_SAC_tiers.csv",index=False)